In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import json
from scrape_goodreads import *

In [ ]:
all_isbn = np.load("book_asin_filter.npy",allow_pickle=True).tolist()

In [ ]:
to_download = all_isbn
to_download = np.array(to_download)
to_download.sort()

In [ ]:
def get_record_from_isbn(isbn):
    
    data = {}
        
    try:
        book_url = get_page("https://www.goodreads.com/book/isbn/{}".format(isbn), data)

        soup = bs(book_url.text, 'html.parser')
        record = get_record(soup)
        record["isbn"] = isbn
        time.sleep(1)
        
    except:
        record = {}
        record["isbn"] = isbn
        
    return record

In [ ]:
from pqdm.processes import pqdm

In [ ]:
# Parallelization
records = pqdm(to_download, get_record_from_isbn, n_jobs=16)